### Import some settings, and modules


In [21]:
import json
from pprint import pp
from ncclient import manager
import xmltodict
import xml.dom.minidom
settings = {}
with open("settings.json") as f:
    settings = json.loads(f.read())
print(settings)

{'host': 'ios-xe-mgmt-latest.cisco.com', 'username': 'developer', 'password': 'C1sco12345', 'netconf_port': 10000}


### Connect to the device

In [33]:
m =  manager.connect(
                    host=settings["host"], 
                    port=settings["netconf_port"], 
                    username=settings["username"], 
                    password=settings["password"], 
                    hostkey_verify=False, 
                    device_params={'name':'iosxe'}
                    )

### Get capabilitys

In [20]:
pp([x for x in m.server_capabilities])

04',
 'urn:ietf:params:xml:ns:yang:ietf-netconf-notifications?module=ietf-netconf-notifications&revision=2012-02-06',
 'urn:ietf:params:xml:ns:yang:ietf-ospf?module=ietf-ospf&revision=2015-03-09&features=ttl-security,te-rid,router-id,remote-lfa,prefix-suppression,ospfv3-authentication-ipsec,nsr,node-flag,multi-topology,multi-area-adj,mtu-ignore,max-lsa,max-ecmp,lls,lfa,ldp-igp-sync,ldp-igp-autoconfig,interface-inheritance,instance-inheritance,graceful-restart,fast-reroute,demand-circuit,bfd,auto-cost,area-inheritance,admin-control&deviations=cisco-xe-ietf-ospf-deviation',
 'urn:ietf:params:xml:ns:yang:ietf-restconf-monitoring?module=ietf-restconf-monitoring&revision=2017-01-26',
 'urn:ietf:params:xml:ns:yang:ietf-routing?module=ietf-routing&revision=2015-05-25&features=router-id,multiple-ribs&deviations=cisco-xe-ietf-routing-deviation',
 'urn:ietf:params:xml:ns:yang:ietf-yang-library?module=ietf-yang-library&revision=2016-06-21',
 'urn:ietf:params:xml:ns:yang:ietf-yang-push?module=ietf

### Get configuration details

In [41]:
# Create an XML filter for targeted NETCONF queries
# This filter get's all interfaces configuration
netconf_filter = """<filter>
                        <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
                            <interface></interface>
                        </interfaces>
                    </filter>"""
netconf_reply = m.get_config(source = 'running', filter = netconf_filter)
print(xml.dom.minidom.parseString(netconf_reply.xml).toprettyxml())

<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:aaf7c025-500e-44d4-a28d-3f4b8942b2c8">
	<data>
		<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
			<interface>
				<name>GigabitEthernet1</name>
				<description>MANAGEMENT INTERFACE - DON'T TOUCH ME</description>
				<type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
				<enabled>true</enabled>
				<ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
					<address>
						<ip>10.10.20.48</ip>
						<netmask>255.255.255.0</netmask>
					</address>
				</ipv4>
				<ipv6 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
			</interface>
			<interface>
				<name>GigabitEthernet2</name>
				<description>Configured with NETCONF 8/23/2020</description>
				<type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
				<enabled>true</enabled>
				<ipv4 xmln

### Add 10 loopbacks based on a template

In [40]:
netconf_interface_template = """
 <config>
     <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
         <interface>
             <name>{name}</name>
             <description>{desc} - Configured with netconf</description>
             <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">
                 {type}
             </type>
             <enabled>{status}</enabled>
             <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
                 <address>
                     <ip>{ip_address}</ip>
                     <netmask>{mask}</netmask>
                 </address>
             </ipv4>
         </interface>
     </interfaces>
 </config>"""

for i in range(200, 211):
    loopback = {}
    loopback["name"] = f"Loopback {i}"
    loopback["desc"] = f"Loopback {i}"
    loopback["type"] = 'ianaift:softwareLoopback'
    loopback["status"] = "true"
    loopback["ip_address"] = f"100.64.{i}.1"
    loopback["mask"] = "255.255.255.255"
    

    loopback_model = netconf_interface_template.format(
         name = loopback["name"],
         desc = loopback["desc"],
         type = loopback["type"],
         status = loopback["status"],
         ip_address = loopback["ip_address"],
         mask = loopback["mask"]
    )
    #print(loopback_model)
    netconf_reply = m.edit_config(loopback_model, target = 'running')
    print(xml.dom.minidom.parseString(netconf_reply.xml).toprettyxml())

<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:25dcd3f8-79cb-41f1-a10d-8cde0d5dd80c">
	<ok/>
</rpc-reply>

<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:f5926760-1130-4fc3-a42e-77ee61730182">
	<ok/>
</rpc-reply>

<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:a8479154-4ee3-457f-a336-efe629e92eb7">
	<ok/>
</rpc-reply>

<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:9c87707c-e692-4890-93fd-831091f34d1b">
	<ok/>
</rpc-reply>

<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id=